## Read Data

In [23]:
import numpy as np
import pandas as pd
import spacy
import io
import glob

filenames = glob.glob('./Data/*txt')
file_name_and_text = {}

for file in filenames:
    with io.open(file, mode="r", encoding="utf-8") as target_file:
         file_name_and_text[file] = target_file.read()

df = pd.DataFrame.from_dict(file_name_and_text, orient='index')
df.reset_index(inplace = True)
df.columns = ['file_name', 'text']
df

,file_name,text
0,./Data\09415616C1 HKR vágott.txt,kibővült az eddigi szolgáltatási összeg az Aeg...
1,./Data\111951940C1 vágott.txt,Üdvözlöm kereslek Jó napot kívánok Én ready fi...
2,./Data\112905428C1 GB vágott.txt,örököltem egy lakást Vagyis te feleségemre van...
3,./Data\113251125C1 LB vágott.txt,Miben segíthetek önnek Érdeklődni szeretnék ho...
4,./Data\123615343C1 Timó vágott.txt,Na vagyok és kaptam kötöttem önöknél egy gépjá...


# Tokenization and Lemmatization with Spacy

### Installing Spacy  
conda install -c conda-forge spacy  
conda install -c conda-forge spacy-lookups-data


### Download Custom Language Model
conda install pip  
pip install https://github.com/oroszgy/spacy-hungarian-models/releases/download/hu_core_ud_lg-0.3.1/hu_core_ud_lg-0.3.1-py3-none-any.whl  


In [24]:
import hu_core_ud_lg
nlp = hu_core_ud_lg.load()

In [27]:
doc = nlp(df.iloc[0,1])

print("{0: <16}".format("Text"), "{0: <14}".format("Lemma"), "PoS", "{0: <6}".format("Dep"), "Head", "IOB", "ENT", "{0: <10}".format("Shape"), "Stop", sep="\t| ")
print("-"*125)
for token in doc:
    print(
        "{0: <16}".format(token.text), "{0: <14}".format(token.lemma_), token.pos_, 
        "{0: <6}".format(token.dep_), token.head.i,
        token.ent_iob_, token.ent_type_,
        "{0: <10}".format(token.shape_), token.is_stop, 
        sep="\t| "
    )

Text            	| Lemma         	| PoS	| Dep   	| Head	| IOB	| ENT	| Shape     	| Stop
-----------------------------------------------------------------------------------------------------------------------------
kibővült        	| kibővül       	| VERB	| ROOT  	| 0	| O	| 	| xxxx      	| False
az              	| az            	| DET	| det   	| 4	| O	| 	| xx        	| True
eddigi          	| eddigi        	| ADJ	| amod  	| 4	| O	| 	| xxxx      	| False
szolgáltatási   	| szolgáltatási 	| ADJ	| amod  	| 4	| O	| 	| xxxx      	| False
összeg          	| összeg        	| NOUN	| nsubj 	| 14	| O	| 	| xxxx      	| False
az              	| az            	| DET	| det   	| 6	| O	| 	| xx        	| True
Aegon           	| Aegon         	| PROPN	| nmod  	| 9	| B	| ORG	| Xxxxx     	| False
orvosi          	| orvosi        	| ADJ	| amod  	| 8	| O	| 	| xxxx      	| False
tápszer         	| tápszer       	| NOUN	| nmod  	| 9	| O	| 	| xxxx      	| False
gyűrű           	| gyűrű         	| NOUN	| nsubj 	

#### Add and Remove Stop Words

In [28]:
nlp.vocab["nem"].is_stop = False
nlp.Defaults.stop_words

{'a',
 'abban',
 'ahhoz',
 'ahogy',
 'ahol',
 'aki',
 'akik',
 'akkor',
 'akár',
 'alatt',
 'amely',
 'amelyek',
 'amelyekben',
 'amelyeket',
 'amelyet',
 'amelynek',
 'ami',
 'amikor',
 'amit',
 'amolyan',
 'amíg',
 'annak',
 'arra',
 'arról',
 'az',
 'azok',
 'azon',
 'azonban',
 'azt',
 'aztán',
 'azután',
 'azzal',
 'azért',
 'be',
 'belül',
 'benne',
 'bár',
 'cikk',
 'cikkek',
 'cikkeket',
 'csak',
 'de',
 'e',
 'ebben',
 'eddig',
 'egy',
 'egyes',
 'egyetlen',
 'egyik',
 'egyre',
 'egyéb',
 'egész',
 'ehhez',
 'ekkor',
 'el',
 'ellen',
 'elo',
 'eloször',
 'elott',
 'elso',
 'elég',
 'előtt',
 'emilyen',
 'ennek',
 'erre',
 'ez',
 'ezek',
 'ezen',
 'ezt',
 'ezzel',
 'ezért',
 'fel',
 'felé',
 'ha',
 'hanem',
 'hiszen',
 'hogy',
 'hogyan',
 'hát',
 'ide',
 'igen',
 'ill',
 'ill.',
 'illetve',
 'ilyen',
 'ilyenkor',
 'inkább',
 'is',
 'ismét',
 'ison',
 'itt',
 'jobban',
 'jó',
 'jól',
 'kell',
 'kellett',
 'keressünk',
 'keresztül',
 'ki',
 'kívül',
 'között',
 'közül',
 'le',
 '

### Preprocessing

In [29]:
# Add custom stop words
# nlp_hu.Defaults.stop_words |= {'', ''} -- add before lemmatization
extra_stops = ['\n ', '\n', 'ugye', 'na', 'lehet', 'ön', 'ok', 'oké', 'ő'] # add after lemmatization

In [30]:
calls = df['text']

tokenized_calls = []
for call in calls:
    doc = nlp(call.lower())
    text = []
    for word in doc:
        if not word.is_stop and not word.is_punct and not word.like_num and word.lemma_ not in extra_stops:
            text.append(word.lemma_)
    tokenized_calls.append(text)

df['text_tokenized'] = tokenized_calls
df

,file_name,text,text_tokenized
0,./Data\09415616C1 HKR vágott.txt,kibővült az eddigi szolgáltatási összeg az Aeg...,"[kibővül, eddigi, szolgáltatási, összeg, aegon..."
1,./Data\111951940C1 vágott.txt,Üdvözlöm kereslek Jó napot kívánok Én ready fi...,"[üdvözl, keres, nap, kíván, ready, fizetés, ka..."
2,./Data\112905428C1 GB vágott.txt,örököltem egy lakást Vagyis te feleségemre van...,"[örököl, lakás, feleségem, lakás, családi, ott..."
3,./Data\113251125C1 LB vágott.txt,Miben segíthetek önnek Érdeklődni szeretnék ho...,"[miben, segíthet, érdeklődik, szeret, kap, lev..."
4,./Data\123615343C1 Timó vágott.txt,Na vagyok és kaptam kötöttem önöknél egy gépjá...,"[kap, köt, gépjármű, felelősségbiztosítás, kap..."


# Vectorizing

#### CountVectorizer 1-gram

In [31]:
from sklearn.feature_extraction.text import CountVectorizer

def dummy_fun(doc):
    return doc

cnts = CountVectorizer(
    analyzer='word',
    tokenizer=dummy_fun,
    preprocessor=dummy_fun,
    token_pattern=None)

cnts.fit(tokenized_calls)
cnts.vocabulary_

{'kibővül': 243,
 'eddigi': 78,
 'szolgáltatási': 448,
 'összeg': 552,
 'aegon': 9,
 'orvosi': 396,
 'tápszer': 485,
 'gyűrű': 170,
 'felügyelet': 146,
 'bizonyára': 47,
 'tetszik': 475,
 'lát': 304,
 'gond': 165,
 'fia': 149,
 'sor': 430,
 'nem': 375,
 'kap': 224,
 'ki': 241,
 'fordul': 157,
 'telefon': 469,
 'ügyelet': 561,
 'kapcsán': 227,
 'probléma': 407,
 'szép': 452,
 'elmagyaráz': 100,
 'hölgy': 193,
 'december': 67,
 'fog': 154,
 'megkap': 318,
 'mennyi': 344,
 'díj': 73,
 'kér': 254,
 'máris': 358,
 'nézemne': 390,
 'haragszik': 175,
 'várakoztató': 515,
 'n50': 365,
 'néz': 389,
 'levelő': 301,
 'tájékoztatás': 483,
 'inka': 206,
 't': 458,
 'legalábbis': 294,
 'vonatkozó': 512,
 'hozzájárul': 187,
 'nehez': 372,
 'szolgáltatás': 447,
 'megígér': 336,
 'megkötött': 323,
 'tök': 490,
 'jóorvosi': 221,
 'postai': 406,
 'út': 556,
 'tud': 479,
 'küld': 278,
 'email': 116,
 'továbbít': 478,
 'ugyanúgy': 497,
 'megfelelő': 313,
 'nagyszerű': 367,
 'nyugodt': 381,
 'elektronikus':

In [32]:
vectors = cnts.transform(tokenized_calls).todense()
vectors.shape

(5, 568)

In [33]:
vectors

matrix([[0, 0, 0, ..., 0, 0, 0],
        [0, 1, 0, ..., 1, 0, 0],
        [0, 2, 0, ..., 0, 0, 0],
        [0, 3, 0, ..., 0, 1, 2],
        [1, 2, 1, ..., 0, 0, 0]], dtype=int64)

In [34]:
vocab = np.array(cnts.get_feature_names())
vocab.shape

(568,)

In [35]:
vocab[350:400]

array(['mindegy', 'mindenképpen', 'mire', 'mond', 'mondható', 'mostani',
       'mostantól', 'motoromm', 'máris', 'márpedig', 'második', 'módosul',
       'módosít', 'módosítás', 'működik', 'n50', 'nagyapa', 'nagyszerű',
       'nagyvilág', 'nap', 'negyedév', 'negyedéves', 'nehez', 'nek',
       'nektaria', 'nem', 'netrisken', 'nincsen', 'nyilván', 'nyolc',
       'nyomtat', 'nyugodt', 'nyújt', 'nál', 'nála', 'nálam', 'négy',
       'négyes', 'név', 'néz', 'nézemne', 'nő', 'olcsó', 'oldal', 'oly',
       'om', 'orvosi', 'os', 'ot', 'otthon'], dtype='<U20')

#### CountVectorizer 2-gram

In [36]:
from sklearn.feature_extraction.text import CountVectorizer

def dummy_fun(doc):
    return doc

cnts_2gram = CountVectorizer(
    analyzer='word',
    ngram_range=(2, 2),
    tokenizer=dummy_fun,
    preprocessor=dummy_fun,
    token_pattern=None)

cnts_2gram.fit(tokenized_calls)
cnts_2gram.vocabulary_

{'kibővül eddigi': 421,
 'eddigi szolgáltatási': 132,
 'szolgáltatási összeg': 821,
 'összeg aegon': 1026,
 'aegon orvosi': 15,
 'orvosi tápszer': 713,
 'tápszer gyűrű': 926,
 'gyűrű felügyelet': 288,
 'felügyelet bizonyára': 226,
 'bizonyára tetszik': 75,
 'tetszik lát': 881,
 'lát gond': 547,
 'gond fia': 275,
 'fia sor': 230,
 'sor nem': 775,
 'nem kap': 663,
 'kap ki': 382,
 'ki fordul': 419,
 'fordul telefon': 261,
 'telefon ügyelet': 870,
 'ügyelet kapcsán': 1047,
 'kapcsán probléma': 398,
 'probléma szép': 738,
 'szép elmagyaráz': 828,
 'elmagyaráz hölgy': 159,
 'hölgy december': 322,
 'december fog': 113,
 'fog kap': 252,
 'kap megkap': 387,
 'megkap mennyi': 570,
 'mennyi díj': 602,
 'díj nem': 127,
 'fordul kér': 259,
 'kér máris': 446,
 'máris nézemne': 629,
 'nézemne haragszik': 702,
 'haragszik várakoztató': 293,
 'várakoztató n50': 968,
 'n50 néz': 638,
 'néz máris': 700,
 'máris levelő': 628,
 'levelő tájékoztatás': 532,
 'tájékoztatás inka': 923,
 'inka t': 351,
 't nem

#### Tf-Idf 1-gram

In [40]:
from sklearn.feature_extraction.text import TfidfVectorizer

def dummy_fun(doc):
    return doc

tfidf = TfidfVectorizer(
    analyzer='word',
    tokenizer=dummy_fun,
    preprocessor=dummy_fun,
    token_pattern=None)

tfidf.fit(tokenized_calls)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=None,
                min_df=1, ngram_range=(1, 1), norm='l2',
                preprocessor=<function dummy_fun at 0x0000020483DC2A60>,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern=None,
                tokenizer=<function dummy_fun at 0x0000020483DC2A60>,
                use_idf=True, vocabulary=None)

In [43]:
tfidf_vectors = tfidf.transform(tokenized_calls).todense()
tfidf_vectors.shape

(5, 568)

## STTM

In [2]:
import sys
sys.executable

'C:\\Users\\poorj\\Projects\\NLP-Tools\\env\\python.exe'

In [1]:
from gsdmm import MovieGroupProcess

In [2]:
mgp = MovieGroupProcess(K=10, alpha=0.1, beta=0.1, n_iters=30)

In [3]:
mgp